# Spotify Insights
##### Equipe: Matheus Francisco e Otávio Moratelli
##### Dataset https://www.kaggle.com/datasets/dhruvildave/spotify-charts
note: The value of streams is NULL when the chart column is "viral50".

1. Carga de um conjunto de dados com um volume de dados adequado para o contexto da
disciplina (+1000 registros);
2. Realização do pré-processamento sobre o conjunto de dados;
3. A partir do conjunto resultante (pré-processado), estatísticas e visualizações devem ser
geradas para promover um entendimento inicial dos dados;
4. Com o conjunto de dados do item 3 aplique algum algoritmo de aprendizado de máquina,
apresente e discuta os resultados obtidos. Talvez seja necessário realizar transformações
para que os dados se adequem ao algoritmo escolhido.
A partir disso, cada equipe deve realizar uma apresentação discutindo todos os itens da
implementação (descrição e carga do conjunto de dados, pré-processamento realizado,
estatísticas e visualizações e, por fim, utilização de um algoritmo de aprendizado de máquina
com apresentação dos resultados).

##### Imports necessários

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import pyspark.sql.functions as f
import pyspark.sql.types as t
import pandas as pd
import time

### Carga de dados com um volume de dados adequado (+26M de registros);
O código abaixo cria o dataframe em chunks de tamanho 100000 para melhorar a performance da carga dos dados.


In [ ]:
spotify_data = pd.read_csv("charts.csv", chunksize=100000)
spotify_data = pd.concat(spotify_data, ignore_index=True)
spotify_data.head(5)

Backup de dados (apenas para desenvolvimento)

In [ ]:
backup = spotify_data
backup.shape

#### Pré-processamento de dados
O código abaixo realiza o pré-processamento dos dados, removendo as colunas que não serão utilizadas e removendo os registros que não possuem valor para a coluna streams.

Também é selecionado um determinado periodo de tempo, neste caso de primeiro de abril de 2020 até o dia 30 de abril de 2020.

In [ ]:
start_date = '2020-04-01'
end_date = '2020-04-30'
mask = spotify_data['date'].between(start_date, end_date)
spotify_data = spotify_data.loc[mask]
spotify_data.shape

#### Remoção de caracteres especiais, mudança nos nomes definidos na coluna trend e eliminação de registros com valores nulos

In [ ]:
import re
spotify_data = spotify_data.replace(re.compile(r'[^\w\s]+'), '', regex=True)
spotify_data['trend'] = spotify_data['trend'].replace('MOVE_DOWN', 0)
spotify_data['trend'] = spotify_data['trend'].replace('MOVE_UP', 1)
spotify_data['trend'] = spotify_data['trend'].replace('SAME_POSITION', 2)
spotify_data['trend'] = spotify_data['trend'].replace('NEW_ENTRY', 3)

missing = spotify_data.isna()
spotify_data = spotify_data.dropna()

##### Definição da região do Brasil

In [ ]:

condition = (spotify_data['region'] == 'Brazil')
brazil = spotify_data.loc[condition]
brazil = brazil.sort_values(by=['rank'] , ascending=True)
brazil.head(5)

# Análise dos dados
#### Top 10 artistas com mais streams no mês de abril de 2020

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import axes3d

brazil_best_up_10 = brazil.loc[brazil['rank'] <= 10]
brazil_best_up_10 = brazil_best_up_10.sort_values(by=['streams'] , ascending=False)
top10 = brazil_best_up_10.groupby('artist').mean()
top10.drop(['rank', 'trend'], axis=1, inplace=True)


top10 = top10.nlargest(10, 'streams')


print(top10)
y = top10['streams']
# z = brazil['rank']
X = top10.index
# plot
plt.figure(figsize=(20, 10))
fig, ax = plt.subplots()
sizes = np.random.uniform(150, 800, len(X))
colors = np.random.uniform(15, 80, len(X))
ax.scatter(X, y, s=sizes, c=colors)

plt.show()


### Streams das musicas dos artistas que passaram no topo da parada

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(figsize=(20,7))
sns.lineplot(x = 'date', y = 'streams', data=brazil_best_up_10, hue='title', ci=None).set_title('Top ')

### Rank da Luiza Sonza

In [ ]:
luiza = brazil.loc[brazil['artist'] == 'Luísa Sonza']

fig, axes = plt.subplots(figsize=(20,7))
sns.lineplot(x = 'date', y = 'rank', data=luiza, hue='title', ci=None).set_title('Rank Da Luiza')

### Streams Gusttavo Lima

In [ ]:
gustavo = brazil.loc[brazil['artist'] == 'Gusttavo Lima']
fig, axes = plt.subplots(figsize=(20,7))
sns.lineplot(x = 'date', y = 'streams', data=gustavo, hue='title', ci=None).set_title('Streams do Gustavo Lima')

# Wordcloud
#### Wordcloud dos artistas mais tocados no mês de abril de 2020

In [ ]:
import matplotlib.pyplot as plt

from wordcloud import WordCloud

def visualize_word_counts(counts):
    wc = WordCloud(max_font_size=130, min_font_size=25, colormap='tab20', background_color='white',
                   prefer_horizontal=.95, width=2100, height=700, random_state=0)
    cloud = wc.generate_from_frequencies(counts)
    plt.figure(figsize=(18,15))
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

counts = brazil['artist'].value_counts()

visualize_word_counts(counts)

# Algoritmo de classificação

In [ ]:
import numpy
import matplotlib.pyplot as plt

x = numpy.random.uniform(brazil_best_up_10['streams'])

plt.hist(x, 100)
plt.show()